In [114]:
import pandas as pd
import numpy as np

In [115]:
movies=pd.read_csv('tmdb-movie-metadata/tmdb_5000_movies.csv')

In [116]:
print (movies.head())
print (movies.shape)

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [117]:
#tfidf of the overview columns
movies['overview'].fillna('NA',inplace=True)
movies['tagline'].fillna('NA',inplace=True)
print (movies[movies['overview'].isna()])

Empty DataFrame
Columns: [budget, genres, homepage, id, keywords, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count]
Index: []


In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=100,max_df=0.7)
X = vectorizer.fit_transform(movies['tagline'])

In [119]:
#print (X.toarray())
#print (vectorizer.get_feature_names())
vecotrizer_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

In [120]:
merged_df = pd.concat([movies[['original_title','genres']],vecotrizer_df], axis=1)

In [121]:
print (merged_df.head())

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                              genres  about  adventure  all  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...    0.0   0.000000  0.0   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...    0.0   0.460691  0.0   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...    0.0   0.000000  0.0   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...    0.0   0.000000  0.0   
4  [{"id": 28, "name": "Action"}, {"id": 12, "nam...    0.0   0.000000  0.0   

    an  and  are   as       at  ...   we  what  when  where  who  will  with  \
0  0.0  0.0  0.0  0.0  0.00000  ...  0.0   0.0   0.0    0.0  0.0   0.0   0.0   
1  0.0  0.0  0.0  0.0  0.45809  ...  0.0   0.0   0.0    0.0  0.0   0.0   0.0   
2 

In [122]:
#First lets dummyfy the genres
import ast
merged_df['genres_list'] = merged_df['genres'].apply(lambda x:ast.literal_eval(x) if len(x)>2 else [{"id": 999, "name": "Unknown"}])


In [123]:
merged_df['genres_list'] = merged_df.loc[:,'genres_list'].apply(lambda x:[d['name'] for d in x])

In [124]:
dummy_genre = pd.get_dummies(merged_df['genres_list'].apply(pd.Series).stack()).sum(level=0)

In [125]:
merged_df = pd.concat([merged_df,dummy_genre], axis=1)

In [126]:
print (merged_df)

                                   original_title  \
0                                          Avatar   
1        Pirates of the Caribbean: At World's End   
2                                         Spectre   
3                           The Dark Knight Rises   
4                                     John Carter   
5                                    Spider-Man 3   
6                                         Tangled   
7                         Avengers: Age of Ultron   
8          Harry Potter and the Half-Blood Prince   
9              Batman v Superman: Dawn of Justice   
10                               Superman Returns   
11                              Quantum of Solace   
12     Pirates of the Caribbean: Dead Man's Chest   
13                                The Lone Ranger   
14                                   Man of Steel   
15       The Chronicles of Narnia: Prince Caspian   
16                                   The Avengers   
17    Pirates of the Caribbean: On Stranger Ti

In [127]:
#merged_df[merged_df.columns=='genres_list']
np.where(merged_df.columns == 'genres_list')[0].item()

102

In [128]:
print (merged_df.iloc[:,2:np.where(merged_df.columns == 'genres_list')[0].item()])

      about  adventure       all        an  and       are   as       at  \
0       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
1       0.0   0.460691  0.000000  0.000000  0.0  0.000000  0.0  0.45809   
2       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
3       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
4       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
5       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
6       0.0   0.516038  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
7       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
8       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
9       0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
10      0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
11      0.0   0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.00000   
12      0.0   0.000000  0

In [136]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(merged_df.iloc[:,2:np.where(merged_df.columns == 'genres_list')[0].item()])
y_kmeans = kmeans.predict(merged_df.iloc[:,2:np.where(merged_df.columns == 'genres_list')[0].item()])

In [137]:
print (y_kmeans)

[2 2 3 ... 0 3 0]


In [138]:
merged_df['cluster']= y_kmeans

In [139]:
print (merged_df)

                                   original_title  \
0                                          Avatar   
1        Pirates of the Caribbean: At World's End   
2                                         Spectre   
3                           The Dark Knight Rises   
4                                     John Carter   
5                                    Spider-Man 3   
6                                         Tangled   
7                         Avengers: Age of Ultron   
8          Harry Potter and the Half-Blood Prince   
9              Batman v Superman: Dawn of Justice   
10                               Superman Returns   
11                              Quantum of Solace   
12     Pirates of the Caribbean: Dead Man's Chest   
13                                The Lone Ranger   
14                                   Man of Steel   
15       The Chronicles of Narnia: Prince Caspian   
16                                   The Avengers   
17    Pirates of the Caribbean: On Stranger Ti

In [140]:
idx=np.where(merged_df.columns == 'genres_list')[0].item()
print (merged_df.groupby(by='cluster').mean()[merged_df.columns[idx+1:idx+21]])

           Action  Adventure  Animation    Comedy     Crime  Documentary  \
cluster                                                                    
0        0.152844   0.082938   0.039100  0.331754  0.081754     0.063981   
1        0.165888   0.098131   0.025701  0.289720  0.156542     0.018692   
2        0.304878   0.243902   0.048780  0.319290  0.134146     0.018847   
3        0.266760   0.183892   0.056331  0.383147  0.173650     0.012570   
4        0.220374   0.130977   0.051975  0.430353  0.137214     0.008316   

            Drama    Family   Fantasy   Foreign   History    Horror     Music  \
cluster                                                                         
0        0.587678  0.090047  0.042654  0.023697  0.052133  0.058057  0.042654   
1        0.474299  0.056075  0.098131  0.000000  0.014019  0.193925  0.051402   
2        0.452328  0.108647  0.110865  0.004435  0.067627  0.105322  0.038803   
3        0.448790  0.120112  0.096834  0.003724  0.033054  0.1

In [141]:
merged_df.columns[np.where(merged_df.columns == 'genres_list')[0].item()+1:]

Index(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
       'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller',
       'Unknown', 'War', 'Western', 'cluster'],
      dtype='object')

In [142]:
print (merged_df[merged_df['cluster']==1])

                        original_title  \
14                        Man of Steel   
32                 Alice in Wonderland   
148                       Ghostbusters   
154              Rise of the Guardians   
164                    Lethal Weapon 4   
174                The Incredible Hulk   
175                            The BFG   
186                        Bad Boys II   
190           The Adventures of Tintin   
194                           Dinosaur   
212             The Day After Tomorrow   
218                       Jason Bourne   
226                    How Do You Know   
238       Teenage Mutant Ninja Turtles   
249                          Insurgent   
257                         Real Steel   
275                    Minority Report   
278                 Planet of the Apes   
297                      Blood Diamond   
336          The Spiderwick Chronicles   
338                          The Alamo   
348     Ice Age: Dawn of the Dinosaurs   
351                       The Depa